# 0. Installing

In [ ]:
! pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread pandas openpyxl

# 1. Adding all events at once

In [2]:
import pandas as pd
import datetime

earnings_sheet = pd.read_excel('Earnings data.xlsx')
just_new_events = earnings_sheet[earnings_sheet['Date']>datetime.datetime.today()]
just_new_events_sorted = just_new_events[::-1]

print(just_new_events_sorted)

   Symbol                      Name       Date       Time              Type
24   HIMS    Hims & Hers Health Inc 2025-05-05  17:00 EST    Earnings Calls
23   HIMS    Hims & Hers Health Inc 2025-05-05        NaN  Earnings Results
22   DDOG               Datadog Inc 2025-05-06  08:00 EST    Earnings Calls
21   DDOG               Datadog Inc 2025-05-06        NaN  Earnings Results
20    SYM              Symbotic Inc 2025-05-07  17:00 EST    Earnings Calls
19    SYM              Symbotic Inc 2025-05-07        NaN  Earnings Results
18   DASH              DoorDash Inc 2025-05-07  17:00 EST    Earnings Calls
17   DASH              DoorDash Inc 2025-05-07        NaN  Earnings Results
16   AFRM       Affirm Holdings Inc 2025-05-08  17:00 EST    Earnings Calls
15   AFRM       Affirm Holdings Inc 2025-05-08        NaN  Earnings Results
14   DKNG            DraftKings Inc 2025-05-08        NaN  Earnings Results
13   BILL         BILL Holdings Inc 2025-05-08  16:30 EST    Earnings Calls
12   BILL   

In [3]:
earnings_sheet_events = just_new_events_sorted.values.tolist()
# Pegando os dois primeiros valores da lista
events = earnings_sheet_events[:4]

# Imprimindo a variável
print(events)

[['HIMS', 'Hims & Hers Health Inc', Timestamp('2025-05-05 00:00:00'), '17:00 EST', 'Earnings Calls'], ['HIMS', 'Hims & Hers Health Inc', Timestamp('2025-05-05 00:00:00'), nan, 'Earnings Results'], ['DDOG', 'Datadog Inc', Timestamp('2025-05-06 00:00:00'), '08:00 EST', 'Earnings Calls'], ['DDOG', 'Datadog Inc', Timestamp('2025-05-06 00:00:00'), nan, 'Earnings Results']]


In [4]:
import os
import pickle
import datetime
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Carregar as credenciais do Google
def get_credentials():
    creds = None
    # O arquivo token.pickle armazena as credenciais do usuário.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # Se as credenciais não estiverem válidas ou não existirem, peça para o usuário se autenticar
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            from google_auth_oauthlib.flow import InstalledAppFlow
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json',  # Arquivo JSON de credenciais obtido na API do Google
                ['https://www.googleapis.com/auth/calendar']
            )
            creds = flow.run_local_server(port=0)
        # Salve as credenciais para futuras execuções
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

# Função para criar evento no Google Calendar
def create_event(service, summary, description, start_date):
    event = {
        'summary': summary,
        'description': description,
        'start': {
            'date': start_date,  # Data do evento
            'timeZone': 'UTC',
        },
        'end': {
            'date': start_date,  # Data final será a mesma para eventos de dia inteiro
            'timeZone': 'UTC',
        },
        'attendees': [
            {'email': 'balacobaco789@gmail.com'},
            # Adicione outros emails de destinatários, se necessário
        ],
    }

    try:
        event = service.events().insert(
            calendarId='primary',
            body=event
        ).execute()
        print(f"Event created: {event.get('htmlLink')}")
    except HttpError as error:
        print(f"Error creating the event: {error}")

def main():
    limit = len(events)
    # Autenticando com o Google Calendar API
    if events[0][0] == events[1][0]:
        skip = True
    else: skip = False

    creds = get_credentials()
    service = build('calendar', 'v3', credentials=creds)
    # Criando eventos para as duas primeiras linhas
    for (index, event_data) in enumerate(events):
        if skip == False:
            symbol, name, date, time, event_type = event_data
            summary = f"{symbol} - {event_type}"
            description = f"Event time: {time}" if type(time)==str else 'No time specified.'

            # Converter datetime para string (usando strftime)
            timestamp_str = date.strftime('%d/%m/%Y')

            # Exibindo o resultado
            print(f"{symbol} - {event_type}: {timestamp_str}")
            
            # A data será utilizada para o evento de dia inteiro
            start_date = datetime.datetime.strptime(timestamp_str, '%d/%m/%Y').date().isoformat()
            print(start_date)

            create_event(service, summary, description, start_date)
        if index != limit-1:
            if (index % 2) != 0: 
                if events[index+1][0] == events[index+2][0]:
                    skip = True
            else: 
                    skip = False

if __name__ == '__main__':
    main()


HIMS - Earnings Results: 05/05/2025
2025-05-05
Event created: https://www.google.com/calendar/event?eid=N290cDdtaHJuanVma3FjNTk2cTM4a3ZpMmMgcmFmYWVsQHhjYXAudmM
DDOG - Earnings Results: 06/05/2025
2025-05-06
Event created: https://www.google.com/calendar/event?eid=dXY4Ymg3NzBiOWpyczhpcG4xN3VwZmRzaTggcmFmYWVsQHhjYXAudmM


# 2. Update events script

#### a. Getting the data

In [ ]:
import pandas as pd
import datetime

earnings_sheet = pd.read_excel('Earnings data.xlsx')
just_new_events = earnings_sheet[earnings_sheet['Date']>datetime.datetime.today()]
just_new_events_sorted = just_new_events[::-1]

print(just_new_events_sorted)

#### b. Prepare the data

In [11]:
earnings_sheet_events = just_new_events_sorted.values.tolist()

# Pegando os dois primeiros valores da lista
events = earnings_sheet_events[:6]

# Imprimindo a variável
print(events)


[['HIMS', 'Hims & Hers Health Inc', Timestamp('2025-05-05 00:00:00'), '17:00 EST', 'Earnings Calls'], ['HIMS', 'Hims & Hers Health Inc', Timestamp('2025-05-05 00:00:00'), nan, 'Earnings Results'], ['DDOG', 'Datadog Inc', Timestamp('2025-05-06 00:00:00'), '08:00 EST', 'Earnings Calls'], ['DDOG', 'Datadog Inc', Timestamp('2025-05-06 00:00:00'), nan, 'Earnings Results'], ['SYM', 'Symbotic Inc', Timestamp('2025-05-07 00:00:00'), '17:00 EST', 'Earnings Calls'], ['SYM', 'Symbotic Inc', Timestamp('2025-05-07 00:00:00'), nan, 'Earnings Results']]


In [6]:
from pandas import Timestamp

# Assuma que esses são seus dois arrays
old_events = events  # array antigo
from pandas import Timestamp

new_events = [
    # HIMS → mantido
    ['HIMS', 'Hims & Hers Health Inc', Timestamp('2025-05-05 00:00:00'), '17:00 EST', 'Earnings Calls'],
    ['HIMS', 'Hims & Hers Health Inc', Timestamp('2025-05-05 00:00:00'), None, 'Earnings Results'],
    
    # DDOG → modificado (+2 dias)
    ['DDOG', 'Datadog Inc', Timestamp('2025-05-08 00:00:00'), '08:00 EST', 'Earnings Calls'],
    ['DDOG', 'Datadog Inc', Timestamp('2025-05-08 00:00:00'), None, 'Earnings Results'],
    
    # SYM → removido (não incluído neste novo array)
]


# Criar conjuntos de identificação baseados na tupla (sigla, timestamp)
old_keys = {(e[0], e[2]) for e in old_events if isinstance(e[2], Timestamp)}
new_keys = {(e[0], e[2]) for e in new_events if isinstance(e[2], Timestamp)}

# print(old_events)
# print(old_keys)
# print(new_keys)

# Encontrar os eventos
same_keys = old_keys & new_keys
removed_keys = old_keys - new_keys
added_keys = new_keys - old_keys

print(same_keys)
print(removed_keys)
print(added_keys)

# Listar os eventos correspondentes
removed_events = [e for e in old_events if (e[0], e[2]) in removed_keys]
added_events = [e for e in new_events if (e[0], e[2]) in added_keys]

print(removed_events)
print(added_events)


{('HIMS', Timestamp('2025-05-05 00:00:00'))}
{('DDOG', Timestamp('2025-05-06 00:00:00')), ('SYM', Timestamp('2025-05-07 00:00:00'))}
{('DDOG', Timestamp('2025-05-08 00:00:00'))}
[['DDOG', 'Datadog Inc', Timestamp('2025-05-06 00:00:00'), '08:00 EST', 'Earnings Calls'], ['DDOG', 'Datadog Inc', Timestamp('2025-05-06 00:00:00'), nan, 'Earnings Results'], ['SYM', 'Symbotic Inc', Timestamp('2025-05-07 00:00:00'), '17:00 EST', 'Earnings Calls'], ['SYM', 'Symbotic Inc', Timestamp('2025-05-07 00:00:00'), nan, 'Earnings Results']]
[['DDOG', 'Datadog Inc', Timestamp('2025-05-08 00:00:00'), '08:00 EST', 'Earnings Calls'], ['DDOG', 'Datadog Inc', Timestamp('2025-05-08 00:00:00'), None, 'Earnings Results']]


In [7]:
from datetime import datetime

def find_closest_event(symbol, old_date, candidate_events):
    candidates = [e for e in candidate_events if e[0] == symbol and isinstance(e[2], Timestamp)]
    if not candidates:
        return None
    closest = min(candidates, key=lambda e: abs((e[2] - old_date).days))
    return closest

events_to_update = []

for old_event in removed_events:
    symbol, _, old_date, *_ = old_event
    closest_new = find_closest_event(symbol, old_date, new_events)
    if closest_new:
        events_to_update.append((old_event, closest_new))  # (original, novo mais próximo)


#### c. Updating existing events function

In [13]:
# -------------------- ATUALIZAÇÃO DE EVENTO --------------------

def update_event(service, old_event, new_event):
    symbol, _, old_date, *_ = old_event
    new_date = new_event[2].date().isoformat()
    summary = f"{symbol} - {new_event[4]}"

    try:
        result = service.events().list(
            calendarId='primary',
            timeMin=old_date.isoformat() + 'T00:00:00Z',
            timeMax=old_date.isoformat() + 'T23:59:59Z',
            q=symbol,
            maxResults=10,
            singleEvents=True,
            orderBy='startTime'
        ).execute()

        for item in result.get('items', []):
            if item['summary'].startswith(symbol):
                item['start']['date'] = new_date
                item['end']['date'] = new_date
                service.events().update(calendarId='primary', eventId=item['id'], body=item).execute()
                print(f"🔁 Evento atualizado: {summary} para {new_date}")
                return

        print(f"⚠ Evento não encontrado para atualização: {summary} em {old_date.date()}")

    except HttpError as error:
        print(f"Erro ao atualizar evento: {error}")

#### d. Creating events function

In [12]:
import os
import pickle
import datetime
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Carregar as credenciais do Google
def get_credentials():
    creds = None
    # O arquivo token.pickle armazena as credenciais do usuário.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # Se as credenciais não estiverem válidas ou não existirem, peça para o usuário se autenticar
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            from google_auth_oauthlib.flow import InstalledAppFlow
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json',  # Arquivo JSON de credenciais obtido na API do Google
                ['https://www.googleapis.com/auth/calendar']
            )
            creds = flow.run_local_server(port=0)
        # Salve as credenciais para futuras execuções
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def create_event(service, symbol, description, date, event_type):
    summary = f"{symbol} - {event_type}"
    body = {
        'summary': summary,
        'description': description,
        'start': {
            'date': date,
            'timeZone': 'UTC',
        },
        'end': {
            'date': date,
            'timeZone': 'UTC',
        },
        'attendees': [
            {'email': 'gustavo@xcap.vc'}
        ],
    }
    try:
        event = service.events().insert(calendarId='primary', body=body).execute()
        print(f"✔ Evento criado: {summary} em {date}")
    except HttpError as error:
        print(f"Erro ao criar evento: {error}")

#### e. Main function

In [18]:
def main():
   # Conectar com API do Google
    creds = get_credentials()
    service = build('calendar', 'v3', credentials=creds)

    # Criar eventos novos
    for e in added_events:
        date_str = e[2].date().isoformat()
        description = f"Event time: {e[3]}" if isinstance(e[3], str) else 'No time specified.'
        create_event(service, e[0], description, date_str, e[4])

    # Atualizar eventos
    for old_e, new_e in events_to_update:
        update_event(service, old_e, new_e)

if __name__ == '__main__':
    main()

✔ Evento criado: DDOG - Earnings Calls em 2025-05-08
✔ Evento criado: DDOG - Earnings Results em 2025-05-08
Erro ao atualizar evento: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?timeMin=2025-05-06T00%3A00%3A00T00%3A00%3A00Z&timeMax=2025-05-06T00%3A00%3A00T23%3A59%3A59Z&q=DDOG&maxResults=10&singleEvents=true&orderBy=startTime&alt=json returned "Bad Request". Details: "[{'domain': 'global', 'reason': 'badRequest', 'message': 'Bad Request'}]">
Erro ao atualizar evento: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?timeMin=2025-05-06T00%3A00%3A00T00%3A00%3A00Z&timeMax=2025-05-06T00%3A00%3A00T23%3A59%3A59Z&q=DDOG&maxResults=10&singleEvents=true&orderBy=startTime&alt=json returned "Bad Request". Details: "[{'domain': 'global', 'reason': 'badRequest', 'message': 'Bad Request'}]">
